In [2]:
import os
import shutil
import warnings
import urllib.request as request
from contextlib import closing

import cupy

import dask_cudf
import dask_ml
import dask

from cuml.manifold import UMAP as cuUMAP
from cuml.dask.decomposition import PCA as cuDaskPCA
from cuml.dask.cluster import KMeans as cuDaskKMeans
from cuml.dask.manifold import UMAP as cuDaskUMAP

from dask.distributed import Client, LocalCluster
from dask_cuda import initialize, LocalCUDACluster
from dask_cuda.local_cuda_cluster import cuda_visible_devices

from bokeh.io.export import export_png
from bokeh.plotting import figure
from bokeh.models.tickers import FixedTicker
from bokeh.io import output_notebook, push_notebook, show

from nvidia.cheminformatics.chembldata import ChEmblData

warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')

output_notebook()

Loading BokehJS ...

### Configurations and settings

In [3]:
# Please add or remove device ids that can be used
# CUDA_VISIBLE_DEVICES=[0]
CUDA_VISIBLE_DEVICES = cuda_visible_devices(0).split(',')

pca_comps = 64
n_clusters = 6
n_neighbors=100
num_mols=10000

enable_tcp_over_ucx = True
enable_nvlink = False
enable_infiniband = False

COLORS = ["#406278", "#e32636", "#9966cc", "#cd9575", "#915c83", "#008000",
          "#ff9966", "#848482", "#8a2be2", "#de5d83", "#800020", "#e97451",
          "#5f9ea0", "#36454f", "#008b8b", "#e9692c", "#f0b98d", "#ef9708",
          "#0fcfc0", "#9cded6", "#d5eae7", "#f3e1eb", "#f6c4e1", "#f79cd4"]
FINGER_PRINT_FILES = 'filter_*.h5'

### Functions

In [4]:
def show_cluster_plot(ldcudf, title='UMAP'):
    """
    Draws a scatter plots from output of UMAP.
    """
    umap_fig = figure(title=title, width=800, output_backend="webgl")
    clusters = ldcudf['cluster'].unique().compute().values_host

    for cluster in clusters:
        query = 'cluster == %s' % (cluster)

        cdf = ldcudf.query(query)
        cdf = cdf.compute()

        if cdf.shape[0] == 0:
            continue

        x_array = cupy.fromDlpack(cdf['x'].to_dlpack())
        y_array = cupy.fromDlpack(cdf['y'].to_dlpack())

        umap_fig.circle(x_array.get(),
                        y_array.get(),
                        size=2,
                        color=COLORS[cluster],
                        alpha=0.5, legend = 'Cluster ' + str(cluster))

    umap_fig.legend.location = 'top_right'
    umap_fig.legend.title = 'Clusters'
    
    umap_fig_handle = show(umap_fig, notebook_handle=True)
    push_notebook(handle=umap_fig_handle)
    

def recluster(tcdf):
    """
    Recluster on a dataset
    """
    tcdf = tcdf.persist()

    print('Reclustering...')
    tcdf = tcdf.drop(['x', 'y', 'cluster'], axis=1)

    print('Computing KMEANS...')
    kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
    kmeans_float.fit(tcdf)
    kmeans_labels = kmeans_float.predict(tcdf)

    print('Computing UMAP...')
    local_model = cuUMAP()
    X_train = tcdf.compute()
    local_model.fit(X_train)

    umap_model = cuDaskUMAP(local_model,
                            n_neighbors=n_neighbors,
                            a=1.0,
                            b=1.0,
                            learning_rate=1.0,
                            client=client)
    Xt = umap_model.transform(tcdf)

    # Add back the column required for plotting and to correlating data
    # between re-clustering
    tcdf['x'] = Xt[0]
    tcdf['y'] = Xt[1]
    tcdf['cluster'] = kmeans_labels
    
    return tcdf

# Download ChEMBL database

In [5]:
data_dir = "/data/db"
db_file = os.path.join(data_dir, 'chembl_27.db')

if not os.path.exists(db_file):
    print('Downloading ChEMBL db...')

    os.makedirs(data_dir, exist_ok=True)
    with closing(request.urlopen('ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_27_sqlite.tar.gz')) as r:
        with open(db_file, 'wb') as f:
            shutil.copyfileobj(r, f)

    print('Download completed')
else:
    print('Reusing available ChEMBL db at', db_file)

Reusing available ChEMBL db at /data/db/chembl_27.db


In [6]:
cluster = LocalCUDACluster(protocol="ucx",
                           dashboard_address=':9001',
                           # TODO: automate visible device list
                           CUDA_VISIBLE_DEVICES=CUDA_VISIBLE_DEVICES,
                           enable_tcp_over_ucx=enable_tcp_over_ucx,
                           enable_nvlink=enable_nvlink,
                           enable_infiniband=enable_infiniband)
client = Client(cluster)
client

Client Scheduler: ucx://127.0.0.1:47825 Dashboard: http://127.0.0.1:36733/status,Cluster Workers: 1 Cores: 1 Memory: 33.27 GB


# Generate fingerprint from ChEMBL

The 4 in ECFP4 corresponds to the diameter of the atom environments considered, while the Morgan fingerprints take a radius parameter. So a Morgan fingerprint with radius=2 is roughly equivalent to ECFP4 and FCFP4.

In [7]:
%%time
cache_directory = '/data/fp'

if cache_directory is None:
    chem_data = ChEmblData(db_file=db_file)
    ddf = chem_data.fetch_all_props(num_recs=num_mols)
else:
    hdf_path = os.path.join(cache_directory, FINGER_PRINT_FILES)
    ddf = dask.dataframe.read_hdf(hdf_path, 'fingerprints')

    if num_mols > 0:
        ddf = ddf.head(num_mols, compute=False, npartitions=-1)

dcudf = dask_cudf.from_dask_dataframe(ddf)
dcudf = dcudf.persist()

# Reduce
pca = cuDaskPCA(client=client, n_components=pca_comps)
dcudf = pca.fit_transform(dcudf)

CPU times: user 8.27 s, sys: 503 ms, total: 8.77 s
Wall time: 17.4 s


# Clustering

In [17]:
%%time

kmeans_float = cuDaskKMeans(client=client, n_clusters=n_clusters)
kmeans_float.fit(dcudf)
kmeans_labels = kmeans_float.predict(dcudf)

CPU times: user 81.9 ms, sys: 7.84 ms, total: 89.8 ms
Wall time: 554 ms


In [18]:
%%time
local_model = cuUMAP()
X_train = dcudf.compute()
local_model.fit(X_train)

umap_model = cuDaskUMAP(local_model,
                        n_neighbors=n_neighbors,
                        a=1.0,
                        b=1.0,
                        learning_rate=1.0,
                        client=client)
Xt = umap_model.transform(dcudf)

dcudf['x'] = Xt[0]
dcudf['y'] = Xt[1]
dcudf['cluster'] = kmeans_labels

dcudf.head(5)

CPU times: user 697 ms, sys: 100 ms, total: 797 ms
Wall time: 1.89 s


,0,1,2,3,4,5,6,7,8,9,...,58,59,60,61,62,63,id_exists,x,y,cluster
0,1.287273,-0.559067,-0.598615,0.650823,1.172983,-0.009365,-1.197538,0.640446,0.717874,-0.052988,...,0.490431,-0.009437,0.083882,0.415819,0.587295,0.153124,False,-1.342994,17.377361,0
1,1.215864,-0.082878,-0.277386,0.027758,1.434051,0.497700,-0.943957,0.110566,0.379220,-0.541240,...,0.521938,0.425353,0.034078,0.532295,0.418451,0.228163,False,-1.383121,17.345627,0
2,1.262036,-0.423914,-0.518359,0.020207,1.526064,-0.973824,-0.221772,0.843168,0.215247,-0.262719,...,-0.133785,0.237246,-0.077575,-0.078481,0.386754,0.065129,True,-1.323973,17.390884,0
3,1.222744,-0.030562,-0.678149,-0.010338,1.017791,0.497684,-0.778859,0.036894,0.658104,-0.638702,...,-0.166094,0.235302,-0.247327,0.426904,0.154290,-0.247629,False,-1.412280,17.315733,0
4,1.319775,-0.270118,-0.573605,-0.022634,1.818706,-0.636643,-1.339001,0.810800,0.700950,-0.104025,...,0.161429,0.236801,-0.038305,0.168600,0.394095,0.191267,True,-1.350735,17.368628,0


In [19]:
molregno = dcudf.index
dcudf['id_exists'] = dcudf.index.isin([2, 4])

df = dcudf.query('id_exists == True')
df = df.compute()

In [20]:
df.index.to_array()

array([2, 4])

In [21]:
%%time
show_cluster_plot(dcudf)

CPU times: user 1.24 s, sys: 67.9 ms, total: 1.31 s
Wall time: 4.9 s


# Re-clustering

In [ ]:
%%time
# Filter data before reclustering
cluster = 1                                         # <- PLEASE CHANGE CLUSTER AS REQUIRED
query = 'cluster == ' + str(cluster)
tcdf = dcudf.query(query)

show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))
tcdf = recluster(tcdf)
print('Plotting UMAP...')
show_cluster_plot(tcdf, title='UMAP - cluster ' + str(cluster))